In [58]:
import pyspark 
from pyspark.sql import SparkSession

In [21]:
pyspark.__version__

'3.3.2'

In [59]:
spark = SparkSession.builder \
        .master('local[*]') \
        .appName('test')\
        .getOrCreate()

In [11]:
pyspark.version

<module 'pyspark.version' from '/home/ramisu/spark/spark-3.3.2-bin-hadoop3/python/pyspark/version.py'>

In [23]:
!ls -lakh code/data/homework/raw


total 62M
drwxrwxr-x 2 ramisu ramisu 4.0K Mar 11 03:40 .
drwxrwxr-x 3 ramisu ramisu 4.0K Mar 11 04:49 ..
-rw-rw-r-- 1 ramisu ramisu  62M Dec 18 21:21 yellow_tripdata_2024-10.parquet


In [24]:
df = spark.read \
    .parquet('code/data/homework/raw/yellow_tripdata_2024-10.parquet')

In [25]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [26]:
df.dtypes

[('VendorID', 'int'),
 ('tpep_pickup_datetime', 'timestamp'),
 ('tpep_dropoff_datetime', 'timestamp'),
 ('passenger_count', 'bigint'),
 ('trip_distance', 'double'),
 ('RatecodeID', 'bigint'),
 ('store_and_fwd_flag', 'string'),
 ('PULocationID', 'int'),
 ('DOLocationID', 'int'),
 ('payment_type', 'bigint'),
 ('fare_amount', 'double'),
 ('extra', 'double'),
 ('mta_tax', 'double'),
 ('tip_amount', 'double'),
 ('tolls_amount', 'double'),
 ('improvement_surcharge', 'double'),
 ('total_amount', 'double'),
 ('congestion_surcharge', 'double'),
 ('Airport_fee', 'double')]

In [27]:
df_repartitioned = df.repartition(4) ## this is a lazy command

In [28]:
df_repartitioned.write.parquet('code/data/homework/partitioned')

In [33]:
from pyspark.sql.functions import col

df_q_3 = df_repartitioned \
    .filter(col("tpep_pickup_datetime").cast("date") == '2024-10-15')

In [35]:
df_q_3.count()

128893

In [36]:
#df_q_3_original = df \
 #   .filter(col("tpep_pickup_datetime").cast("date") == '2024-10-15')
df_q_3_original = df.where("DATE(tpep_pickup_datetime) = '2024-10-15'")

In [37]:
df_q_3_original.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-15 07:31:31|  2024-10-15 07:40:59|              1|         2.24|         1|                 N|          90|         231|           1|       12.1|  0.0|    0.5|      4.0

In [38]:
df_q_3_original.count()

128893

In [39]:
df_repartitioned.createOrReplaceTempView('trips_repartitioned')
df.createOrReplaceTempView('trips')

In [41]:
query = """
SELECT * FROM trips 
WHERE TO_DATE(FROM_UTC_TIMESTAMP(tpep_pickup_datetime, 'Europe/Berlin')) = '2024-10-15'
"""

In [43]:
df_filtered = spark.sql(query)
df_filtered.count()

125567

In [44]:
query_repartitioned = """
SELECT * FROM trips_repartitioned 
WHERE TO_DATE(FROM_UTC_TIMESTAMP(tpep_pickup_datetime, 'Europe/Berlin')) = '2024-10-15'
"""

In [45]:
df_filtered_repartitioned = spark.sql(query_repartitioned)
df_filtered_repartitioned.count()

125567

In [56]:
query_longest_trip = """
SELECT 
    tpep_pickup_datetime as start,
    tpep_dropoff_datetime as end,
    UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime) AS difference,
    (UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 3600 AS difference_hrs,
    (UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / (3600*12) AS difference_days
FROM trips 
ORDER BY 3 desc
LIMIT 100
"""
df_longest = spark.sql(query_longest_trip)
df_longest.show()

+-------------------+-------------------+----------+------------------+------------------+
|              start|                end|difference|    difference_hrs|   difference_days|
+-------------------+-------------------+----------+------------------+------------------+
|2024-10-16 13:03:49|2024-10-23 07:40:53|    585424|162.61777777777777|13.551481481481481|
|2024-10-03 18:47:25|2024-10-09 18:06:55|    515970|           143.325|          11.94375|
|2024-10-22 16:00:55|2024-10-28 09:46:33|    495938|137.76055555555556|11.480046296296296|
|2024-10-18 09:53:32|2024-10-23 04:43:37|    413405|114.83472222222223| 9.569560185185185|
|2024-10-21 00:36:24|2024-10-24 18:30:18|    323634| 89.89833333333333| 7.491527777777778|
|2024-10-20 13:30:52|2024-10-24 06:57:38|    322006| 89.44611111111111| 7.453842592592593|
|2024-10-22 16:04:52|2024-10-25 14:22:49|    253077| 70.29916666666666| 5.858263888888889|
|2024-10-12 19:32:51|2024-10-15 15:07:15|    243264| 67.57333333333334| 5.631111111111111|

In [57]:
query_longest_trip_repartitioned = """
SELECT 
    tpep_pickup_datetime as start,
    tpep_dropoff_datetime as end,
    UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime) AS difference,
    (UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 3600 AS difference_hrs,
    (UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / (3600*12) AS difference_days
FROM trips_repartitioned 
ORDER BY 3 desc
LIMIT 100
"""
df_longest_repartitioned = spark.sql(query_longest_trip_repartitioned)
df_longest_repartitioned.show()

+-------------------+-------------------+----------+------------------+------------------+
|              start|                end|difference|    difference_hrs|   difference_days|
+-------------------+-------------------+----------+------------------+------------------+
|2024-10-16 13:03:49|2024-10-23 07:40:53|    585424|162.61777777777777|13.551481481481481|
|2024-10-03 18:47:25|2024-10-09 18:06:55|    515970|           143.325|          11.94375|
|2024-10-22 16:00:55|2024-10-28 09:46:33|    495938|137.76055555555556|11.480046296296296|
|2024-10-18 09:53:32|2024-10-23 04:43:37|    413405|114.83472222222223| 9.569560185185185|
|2024-10-21 00:36:24|2024-10-24 18:30:18|    323634| 89.89833333333333| 7.491527777777778|
|2024-10-20 13:30:52|2024-10-24 06:57:38|    322006| 89.44611111111111| 7.453842592592593|
|2024-10-22 16:04:52|2024-10-25 14:22:49|    253077| 70.29916666666666| 5.858263888888889|
|2024-10-12 19:32:51|2024-10-15 15:07:15|    243264| 67.57333333333334| 5.631111111111111|

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

In [95]:
df_lookup = spark.read \
    .option("header","true")\
    .csv('code/data/homework/raw/taxi_zone_lookup.csv')

In [96]:
df_lookup.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [97]:
df_lookup.createOrReplaceTempView('lookup_table')


In [98]:
simple_query = """
SELECT * FROM lookup_table
"""
df_simple = spark.sql(simple_query)
df_simple.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [100]:
query_least_frequent = """
SELECT 
    p.pickup_location_id,
    l.Borough,
    l.Zone,
    l.service_zone
FROM(
SELECT 
    PULocationID as pickup_location_id,
    COUNT(PULocationID) AS count_pu_location
FROM trips 
WHERE YEAR(tpep_pickup_datetime) = 2024
AND MONTH(tpep_pickup_datetime) = 10
GROUP BY PULocationID
ORDER BY 2 asc
limit 1
)  p
LEFT JOIN lookup_table l ON l.LocationID = p.pickup_location_id
"""
df_least = spark.sql(query_least_frequent)
df_least.show()

+------------------+---------+--------------------+------------+
|pickup_location_id|  Borough|                Zone|service_zone|
+------------------+---------+--------------------+------------+
|               105|Manhattan|Governor's Island...| Yellow Zone|
+------------------+---------+--------------------+------------+



In [101]:
df_least.write.csv('code/data/homework/answer/least.csv')